# Setting up the Notebook

In [2]:
import sys
sys.path.append('/home/clachris/Documents/projects/DIET_SSL/py_files')
print(sys.path)
from py_files.models import *
from py_files.dataset_organization import *
from py_files.train_pretext import *
from full_train_pretext import *

['/home/clachris/Documents/projects/Diet_SSL', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/clachris/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/home/clachris/Documents/projects/DIET_SSL/py_files', '/home/clachris/Documents/projects/DIET_SSL/py_files']


# Setting up the data

## Dysis Cervices

In [3]:
# Reading in and checking the Dysis image paths

dysis_df = pd.read_csv('Dysis_Cervix/dysis_cervix_images_paths.csv')
dysis_df = dysis_df.drop('Unnamed: 0', axis = 1)
dysis_df.head()

dysis_df = dysis_df.stack(-1)
dysis_list = list(dysis_df)
print('Confirming this looks like what we want:', dysis_list[:20])


Confirming this looks like what we want: ['/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img000.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img007.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img014.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img021.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img028.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img035.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img042.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00017/00017_img049.bmp', '/home/clachris/Documents/datasets/cervix/Dysis_Cervix/JayashreeSample/lesion_set_1/00

In [20]:
# Getting the image paths of the resized images

root_dir = 'IRIS_Cervix/IRIS_Resized/'

X, Y = create_X_Y(root_dir, -1, True)

Lengths of our X and Y (199, 199)


['IRIS_Cervix/IRIS_Resized/GYFZ-00000017.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000033.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000055.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000056.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000083.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000084.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000095.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000156.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000264.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000268.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000269.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000307.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000325.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000352.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000361.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000362.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000363.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000402.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000432.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000433.png',
 'IRIS_Cervix/IRIS_Resized/GYFZ-00000476.png',
 'IRIS_Cervix

# Brief description of the problem and architecture

In supervised learning, we are given a set of labels $Y$ to go along with data $X$. The idea is then to learn some function $\hat{f}_{\theta}$ such that:

$$ \hat{f}_{\theta}: X \rightarrow Y \quad \mathbf{x}_i \rightarrow y_i $$

However, we often don't have labels, or perhaps *enough* labels. Instead, we would like to train a model to understand salient features of the data so that fewer labels are sufficient, as we already understand the data well before even trying to classify.

The *DIET*, meaning Datum IndEx as it Target, employs the datum index as it's target class. As in, given a datset of $N$ samples $\{\mathbf{x}_1, \ldots, \mathbf{x{}_N}\}$, define the class of $\mathbf{x}_n$ as simply $n$. This means the loss function is:

$$ \mathcal{L}_{DIET}(\mathbf{x}_n) = \mathcal{L}_{Classification}(W f_{\theta}(\mathbf{x}_n), n) $$

Where $\mathbf{x}_n \in \mathbb{R}^D$ is the datum, $f_{\theta}: \mathbb{R}^D \rightarrow \mathbb{R}^K$ is the neural network, $W \in \mathbb{R}^{K \times N}$ is the classifier, and $\mathcal{L}_{Classification}$ is the preferred classification loss. 

# Building the network

In [6]:
# Setting up the device

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# GPU types supported
print(torch.cuda.get_arch_list())

# Cuda version
print(torch.version.cuda)

cuda
['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']
11.3


In [7]:
# Model Instantiation

model = DIET_UNet(spatial_dims = 2,
in_channels = 3,
out_channels = 1,
channels = (8, 16, 32, 64),
strides = (2, 2, 2),
norm = 'batch',
dropout = 0,
num_classes=len(Y))

In [16]:
# Organizing the data

train_X, train_Y, val_X, val_Y, test_X, test_Y = DataDivision(X, Y, 0.8, 0.2)

In [17]:
# Datasets

train_dataloader = Dataset_Dataloader(train_X, train_Y, 2)
val_dataloader = Dataset_Dataloader(val_X, val_Y, 2)
test_dataloader = Dataset_Dataloader(test_X, test_Y, 2)

In [18]:
# Confirming everything has the shape we need

tr_c = conf(train_dataloader, True)
val_c = conf(val_dataloader, True)
te_c = conf(test_dataloader, True)

The number of batches and the shapes of the first and second items in the dataloader batch are: 80 (2, 3, 512, 512) (2,)
The number of batches and the shapes of the first and second items in the dataloader batch are: 10 (2, 3, 512, 512) (2,)
The number of batches and the shapes of the first and second items in the dataloader batch are: 10 (2, 3, 512, 512) (2,)


# Training

In [ ]:
# Training

trainer = Train(model,
                10,
                train_dataloader, 
                val_dataloader, 
                1e-3, 
                'None',
                'Dysis_Cervix/', 
                False,
                False,
                False)

trainer.configure_devices('0')

trainer.fit()